In [6]:
import os
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import Ridge
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import StackingRegressor
from sklearn.multioutput import MultiOutputRegressor
import joblib

# Set global font settings for plots
plt.rcParams['font.sans-serif'] = ['Arial']
plt.rcParams['axes.unicode_minus'] = False

# Transformer to squeeze 3D inputs (e.g., [n, m, 1]) to 2D
class SqueezeTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None): return self
    def transform(self, X):
        if isinstance(X, tuple): X = X[0]
        if hasattr(X, "ndim") and X.ndim == 3 and X.shape[-1] == 1:
            return np.squeeze(X, axis=-1)
        return X

# Custom weighted RMSE computation
def compute_weighted_rmse(y_true, y_pred, elements, weights_dict):
    total_rmse = total_w = 0.0
    for i, elem in enumerate(elements):
        rmse_i = np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i]))
        w = weights_dict.get(elem, 1.0)
        total_rmse += rmse_i * w
        total_w += w
    return total_rmse / total_w if total_w else np.nan

# Scorer for weighted RMSE, to be used in GridSearchCV
def get_weighted_rmse_scorer(file_elements, weights_dict):
    def scorer(estimator, X, y):
        y_pred = estimator.predict(X)
        return -compute_weighted_rmse(y, y_pred, file_elements, weights_dict)
    return scorer

# Plot predicted vs actual values

def plot_actual_vs_predicted(y_true, y_pred, element_name, season_name, save_path=None):
    plt.figure(figsize=(6, 6))
    plt.scatter(y_true, y_pred, alpha=0.6)
    plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', linewidth=2)
    plt.xlabel(f"Actual {element_name} (Test)")
    plt.ylabel(f"Predicted {element_name} (Test)")
    plt.title(f"{season_name} - {element_name} (n_test={len(y_true)})")
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path)
    plt.close()

# Element-specific weights
WEIGHTS_DICT = {
    "Nitrogen": 3.0, "Phosphorus": 3.0, "Potassium": 3.0,
    "Boron": 2.0, "Manganese": 2.0
}

# Define spectral range
WAVELENGTH_START = 400
WAVELENGTH_END = 2500

# Process spectral columns and optionally bin wavelengths

def process_spectral_columns_dynamic(df, apply_binning=True):
    wavelength_cols = sorted(
        [c for c in df.columns if re.match(r"^\d+\.?\d*nm$", c)],
        key=lambda x: float(x[:-2])
    )
    if not wavelength_cols:
        raise ValueError("No spectral columns found")
    wavelengths = np.array([float(c[:-2]) for c in wavelength_cols])
    if not apply_binning:
        return df.copy(), wavelengths

    target_wls = np.arange(WAVELENGTH_START, WAVELENGTH_END + 1, 1)
    col_data = {}
    for target in target_wls:
        idx = np.where((wavelengths >= target - 0.5) & (wavelengths <= target + 0.5))[0]
        if idx.size:
            bins = [wavelength_cols[i] for i in idx]
            col_data[f"{int(target)}nm"] = df[bins].mean(axis=1)
        else:
            col_data[f"{int(target)}nm"] = np.nan
    proc = pd.DataFrame(col_data, index=df.index)
    nonspec = df.columns.difference(wavelength_cols)
    return pd.concat([df[nonspec], proc], axis=1), target_wls

# Main modeling pipeline

def run_model(
    imputed_data_root="data/DSMP_Prob_Dalhousie_2025 - data/imputed_data",
    apply_binning=True,
    use_grid_search=True,
    strategy="PLS_Ridge_Stack"
):
    results_dir = Path("./results") / strategy
    results_dir.mkdir(parents=True, exist_ok=True)

    subfolders = ["DRIED", "FRESH"]
    all_elements = [
        "Nitrogen", "Phosphorus", "Potassium", "Calcium", "Magnesium",
        "Sulfur", "Manganese", "Boron", "Zinc", "Iron", "Copper",
        "Aluminum", "Sodium", "Chloride"
    ]
    all_metrics = []

    for season_type in subfolders:
        data_dir = Path(imputed_data_root) / season_type
        for idx, file in enumerate(sorted(data_dir.glob("*.csv"))):
            print(f"\n==== Processing {season_type} file: {file.name} ====")
            df = pd.read_csv(file)
            processed_df, wavelengths = process_spectral_columns_dynamic(df, apply_binning)
            wl_cols = [f"{int(w)}nm" for w in wavelengths]

            file_elements = [e for e in all_elements if e in processed_df.columns]
            if not file_elements:
                print("  No target elements found. Skipping.")
                continue

            valid = processed_df.dropna(subset=file_elements)
            X, Y = valid[wl_cols].values, valid[file_elements].values
            if X.shape[0] < 10:
                print("  Too few samples. Skipping.")
                continue

            X_tr, X_te, Y_tr, Y_te = train_test_split(X, Y, test_size=0.2, random_state=42)
            print(f"  Train/Test split — Train: {len(X_tr)}, Test: {len(X_te)}")

            if strategy == "PLS_Ridge_Stack":
                pls_pipe = Pipeline([('scaler', StandardScaler()), ('pls', PLSRegression())])
                ridge_pipe = Pipeline([('scaler', StandardScaler()), ('ridge', Ridge())])
                stack = StackingRegressor(
                    estimators=[('pls', pls_pipe), ('ridge', ridge_pipe)],
                    final_estimator=Ridge(), passthrough=False, n_jobs=-1
                )
                multi_stack = MultiOutputRegressor(stack, n_jobs=-1)
                model = Pipeline([
                    ('x_scaler', StandardScaler()),
                    ('model', TransformedTargetRegressor(
                        regressor=multi_stack, transformer=StandardScaler()
                    ))
                ])
                param_grid = {
                    'model__regressor__estimator__pls__pls__n_components': [5, 10, 15],
                    'model__regressor__estimator__ridge__ridge__alpha': [0.1, 1, 10],
                    'model__regressor__estimator__final_estimator__alpha': [0.1, 1, 10]
                }
            else:
                raise ValueError("Unknown strategy")

            if use_grid_search:
                scorer = get_weighted_rmse_scorer(file_elements, WEIGHTS_DICT)
                gs = GridSearchCV(model, param_grid=param_grid, scoring=scorer,
                                  cv=5, n_jobs=-1, verbose=1)
                gs.fit(X_tr, Y_tr)
                best = gs.best_estimator_
                print("  Best parameters:", gs.best_params_)
                print(f"  Weighted RMSE on training = {-gs.best_score_:.4f}")
            else:
                best = model.fit(X_tr, Y_tr)

            Y_pred = best.predict(X_te)
            metrics = []

            subdir = results_dir / season_type / f"file_{idx+1}"
            subdir.mkdir(parents=True, exist_ok=True)

            for j, elem in enumerate(file_elements):
                rmse = np.sqrt(mean_squared_error(Y_te[:, j], Y_pred[:, j]))
                r2 = r2_score(Y_te[:, j], Y_pred[:, j])
                std = np.std(Y_te[:, j], ddof=1)
                rpd = std / rmse if rmse else np.nan
                metrics.append({
                    "Season": f"{season_type}_{file.stem}",
                    "Strategy": strategy,
                    "Element": elem,
                    "RMSE": round(rmse, 4),
                    "R2": round(r2, 4),
                    "RPD": round(rpd, 4),
                    "n_test": len(X_te)
                })

                if elem in ["Nitrogen", "Phosphorus", "Potassium"]:
                    plot_path = subdir / f"scatter_{elem}.png"
                    plot_actual_vs_predicted(
                        Y_te[:, j], Y_pred[:, j],
                        element_name=elem,
                        season_name=f"{season_type}_{file.stem}",
                        save_path=plot_path
                    )

            dfm = pd.DataFrame(metrics)
            print(dfm.set_index("Element")[["RMSE", "R2", "RPD"]])

            all_metrics.append(dfm)
            dfm.to_csv(subdir / f"metrics_{strategy}.csv", index=False)
            joblib.dump(best, subdir / f"{strategy}_pipeline.pkl")

    # Save all metrics
    if all_metrics:
        combined = pd.concat(all_metrics, ignore_index=True)
        combined.to_csv(results_dir / f"combined_metrics_{strategy}.csv", index=False)
        print(f"\n[{strategy}] All detailed results saved to {results_dir}")

        # Save summary overview (only for selected elements)
        target_elements = ["Nitrogen", "Phosphorus", "Potassium"]
        summary_rows = []
        for season in combined['Season'].unique():
            season_data = combined[combined['Season'] == season]
            weighted_rmse = season_data.apply(
                lambda row: row['RMSE'] * WEIGHTS_DICT.get(row['Element'], 1.0), axis=1
            ).sum() / sum(WEIGHTS_DICT.get(row['Element'], 1.0) for idx, row in season_data.iterrows())

            npk_data = season_data[season_data['Element'].isin(target_elements)]
            avg_rpd_npk = npk_data['RPD'].mean()

            summary_rows.append({
                "Season": season,
                "Weighted_RMSE": round(weighted_rmse, 4),
                "Avg_RPD_NPK": round(avg_rpd_npk, 4)
            })

        summary_df = pd.DataFrame(summary_rows)
        summary_save_path = results_dir / f"summary_overview_{strategy}.csv"
        summary_df.to_csv(summary_save_path, index=False)
        print(f"\n✅ Summary overview saved to {summary_save_path}")
        print(summary_df)

if __name__ == "__main__":
    run_model(apply_binning=True, use_grid_search=True, strategy="PLS_Ridge_Stack")


==== Processing DRIED file: DRIED_season1_imputed.csv ====
  Train/Test split — Train: 32, Test: 8
Fitting 5 folds for each of 27 candidates, totalling 135 fits
  Best parameters: {'model__regressor__estimator__final_estimator__alpha': 0.1, 'model__regressor__estimator__pls__pls__n_components': 10, 'model__regressor__estimator__ridge__ridge__alpha': 10}
  Weighted RMSE on training = 42.1363
                RMSE      R2     RPD
Element                             
Nitrogen      0.3267  0.8214  2.5298
Phosphorus    0.0741 -0.1116  1.0140
Potassium     0.8527  0.8157  2.4899
Calcium       0.0492  0.9819  7.9384
Magnesium     0.0797  0.9322  4.1058
Sulfur        0.0293  0.1039  1.1293
Manganese   203.6068  0.4950  1.5043
Boron         3.9800 -0.3793  0.9103
Zinc         47.1111 -0.2281  0.9647
Iron         67.2560  0.5578  1.6075
Copper        4.2551 -0.2720  0.9479
Aluminum    112.4904  0.4949  1.5041
Sodium        0.0103 -0.0642  1.0363
Chloride      0.4083  0.6805  1.8914

==== Process